Reading in the data from stat.gov:

In [362]:
import xmltodict
import requests
import re
import json
import pandas as pd


def get_dataset_ids(list_of_table_names):
    r = requests.get("https://osp-rs.stat.gov.lt/rest_xml/dataflow/")
    metadata_dict = dict(xmltodict.parse(r.text))
    dataset_ids = []
    
    for i in metadata_dict["mes:Structure"]["mes:Structures"]["str:Dataflows"]["str:Dataflow"]:
        matches = []
        for j in list_of_table_names:
            j = j.lower()
            j = re.sub(r'[^\w\s]','',j)
            j = re.sub(' +', ' ',j)
            name = i["com:Name"][0]["#text"].lower()
            name = re.sub(r'[^\w\s]','',name)
            name = re.sub(' +', ' ',name)
            matches.append(j in name)
        if any(matches):
            dataset_ids.append(i["@id"])
            
    return dataset_ids

def construct_links(dataset_ids,start_period="2015-01"):
    links = []
    link_beginning = "https://osp-rs.stat.gov.lt/rest_json/data/"

    for i in dataset_ids:
        links.append(link_beginning + i + "/?startPeriod=" + start_period)
    
    return links

In [363]:
miestai = ["Kauno m. sav.","Vilniaus m. sav.","Klaipėdos m. sav.","Šiaulių m. sav.","Panevėžio m. sav.","Alytaus m. sav."]

def dataset_to_dataframe(link):
    r = requests.get(link)
    json_dict = r.json()
    skip = False
    
    if not json_dict["dataSets"][0]["observations"]:
        skip = True
    for i in json_dict["structure"]["attributes"]["dataSet"]:
        if i["id"] == "DS_REGIONAL":
            if i["values"][0]["id"]=="N":
                skip = True
        if i["id"] == "DS_TIME_FORMAT":
            if i["values"][0]["name"] not in ["Metai","Mokymo metai"]:
                skip = True
        if i["id"] == "LAIKOTARPIS":
            year_list = []
            for j in i["values"]:
                year_list.append(int(j["name"][0:4]))
            if max(year_list) < 2020:
                skip  = True

    if not skip:
        dimensions_dict = {}

        for i in json_dict["structure"]["dimensions"]["observation"]:
            dimensions_dict[i["name"]] = i["keyPosition"]

            
        list_of_mappings = []

        for i in json_dict["structure"]["dimensions"]["observation"]:
            mapping_dict = {}
            for ind,j in enumerate(i["values"]):
                mapping_dict[str(ind)] = j["name"]
            list_of_mappings.append(mapping_dict)   

        rows = []

        for i in json_dict["dataSets"][0]["observations"].keys():
            rows.append(i.split(":"))

        df = pd.DataFrame(rows)
        for i, j in enumerate(list_of_mappings):
            df[i].replace(j,inplace=True)

        df.columns = dimensions_dict.keys()

        
        values = []

        for i in json_dict["dataSets"][0]["observations"].values():
            values.append(i[0])
        
        df["Reikšmė"]=pd.Series(values)

        df=df[df["Administracinė teritorija"].isin(miestai)]
        
        df.name = json_dict["structure"]["name"]
        
        return df
    else:
        print("Skipped")

In [364]:
table_names = ["Teršalų, išmestų į aplinkos orą iš stacionarių taršos šaltinių","Ūkio, buities ir gamybos nuotekų išleidimas į paviršinius vandenis",
              "Vandens sunaudojimas","Autobusų rida","Vidutiniškai vienam gyventojui tenka kelionių autobusais"]

table_ids=get_dataset_ids(table_names)

table_ids

['S1R078_M8010302',
 'S1R079_M8010301',
 'S5R003_M7030327_3',
 'S5R003_M7030211_3',
 'S5R087_M7030320_1',
 'S1R121_M9070335_1',
 'S1R121_M9070335_2']

In [365]:
links=construct_links(table_ids)

df_list = []
for i in links:
    df_list.append(dataset_to_dataframe(i))
    
df_list = [i for i in df_list if i is not None]

Skipped
Skipped


In [368]:
table_names = ["Nuolatinių gyventojų skaičius liepos 1 d."]
table_ids=get_dataset_ids(table_names)

links=construct_links(table_ids)

population_df = dataset_to_dataframe(links[0])

In [369]:
population_df = population_df[population_df.iloc[:,1] == "Miestas ir kaimas"].iloc[:,[0,3,4]]

In [370]:
df_list_population = []
for i in df_list:
    x = pd.merge(i,population_df,on=["Laikotarpis","Administracinė teritorija"])
    x["per_thousand"]= x["Reikšmė_x"]/x["Reikšmė_y"]*1000
    x = x.replace("Klaipėdos m. sav.", "Klaipėda").replace("Kauno m. sav.", "Kaunas").replace("Vilniaus m. sav.", "Vilnius").replace("Panevėžio m. sav.", "Panevėžys").replace("Šiaulių m. sav.", "Šiauliai").replace("Alytaus m. sav.", "Alytus")
    x.drop(["Matavimo vienetai"], inplace = True,axis=1)
    df_list_population.append(x)

Individually modifying the data:

In [371]:
for i,j in enumerate(df_list_population):
    if i != 3:
        j.drop(["Reikšmė_x","Reikšmė_y"],axis=1,inplace=True)
        j.rename(columns={"Administracinė teritorija": "city","Laikotarpis":"x","per_thousand":"y"},inplace=True)

In [372]:
df_list_population[3] = df_list_population[3][df_list_population[3].iloc[:,1]=="Autobusai"]

df_list_population[3].drop(columns=["Reikšmė_y","per_thousand","Transporto rūšis (Autobusai)"],inplace=True)
df_list_population[3] = df_list_population[3].round(1)
df_list_population[3].rename(columns={"Administracinė teritorija": "city","Laikotarpis":"x","Reikšmė_x":"y"},inplace=True)

In [373]:
df_list_population[3].to_csv("modified/kelioniu_kiekis.csv",index=False)

In [374]:
grouped=df_list_population[4][df_list_population[4]["Teršalai"].isin(["Azoto oksidai, tonos","Anglies monoksidas, tonos","Kietosios medžiagos"])].round(1).groupby("Teršalai")

l=[grouped.get_group(x) for x in grouped.groups]
l=[i.iloc[:,[0,2,3]] for i in l]

In [375]:
l[0].to_csv("modified/tersalai_co.csv",index=False)
l[1].to_csv("modified/tersalai_no.csv",index=False)
l[2].to_csv("modified/tersalai_kietosios.csv",index=False)

In [376]:
grouped=df_list_population[1][df_list_population[1].iloc[:,0].isin(["Sunaudota požeminio vandens","Sunaudota vandens ūkio ir buities reikmėms"])].round(1).groupby("Vandens naudojimo paskirtis")

l=[grouped.get_group(x) for x in grouped.groups]
l=[i.iloc[:,[1,2,3]] for i in l]

In [377]:
l[1].to_csv("vandens_sunaudojimas_pozeminis.csv",index=False)
l[0].to_csv("vandens_sunaudojimas_buiciai.csv",index=False)

In [379]:
df_list_population[2] = df_list_population[2][df_list_population[2]["Reiso tipas"]=="Reguliarus reisas"].iloc[:,[1,2,3]].round(1)
df_list_population[2].to_csv("modified/viesojo_rida.csv",index=False)

In [380]:
nuotekos = df_list_population[0]
nuotekos = pd.merge(
    nuotekos[nuotekos["Išvalymas"] == "Išleista išvalytų iki normos nuotekų"],
    nuotekos[nuotekos["Išvalymas"] == "Iš viso išleista nuotekų"],
    on=["city", "x"])
nuotekos["y"] = round(100*nuotekos["y_x"]/nuotekos["y_y"],2)
nuotekos = nuotekos.drop(['Išvalymas_x', "Išvalymas_y", "y_x", "y_y"], 1)
nuotekos.to_csv("modified/nuotekos.csv", index = False)

C:\Users\dovyd\AppData\Local\Temp/ipykernel_9784/4153598586.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  nuotekos = nuotekos.drop(['Išvalymas_x', "Išvalymas_y", "y_x", "y_y"], 1)


Waste data from eurostat:

In [246]:
import numpy as np
waste = pd.read_csv("csv/urb_cenv_1_Data.csv",encoding = "ISO-8859-1",na_values=":")

In [247]:
miestai = ["Klaipeda","Siauliai","Vilnius","Kaunas","Panevezys","Alytus"]

In [248]:
waste = waste[waste["CITIES"].isin(miestai)][(waste["TIME"]>=2015) & (waste["TIME"]<2020)].iloc[:,[0,1,3]]

C:\Users\dovyd\AppData\Local\Temp/ipykernel_9784/1620741886.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  waste = waste[waste["CITIES"].isin(miestai)][(waste["TIME"]>=2015) & (waste["TIME"]<2020)].iloc[:,[0,1,3]]


In [249]:
l = []
for i in waste["Value"]:
    l.append(float(i))

waste["Value"] = l

In [250]:
waste.rename(columns={"TIME":"x","CITIES":"city","Value":"y"},inplace=True)
waste.replace({"Panevezys":"Panevėžys","Siauliai":"Šiauliai","Klaipeda":"Klaipėda"},inplace=True)

In [241]:
population_df = population_df.replace("Klaipėdos m. sav.", "Klaipėda").replace("Kauno m. sav.", "Kaunas").replace("Vilniaus m. sav.", "Vilnius").replace("Panevėžio m. sav.", "Panevėžys").replace("Šiaulių m. sav.", "Šiauliai").replace("Alytaus m. sav.", "Alytus")
population_df.rename(columns={"Administracinė teritorija":"city","Laikotarpis":"x","Reikšmė":"y"},inplace=True)
population_df["x"]=pd.to_numeric(population_df["x"])

In [251]:
waste = pd.merge(waste,population_df,on=["x","city"])
waste["y"]= waste["y_x"]/waste["y_y"]*1000

waste = waste.loc[:,["x","city","y"]]
waste.dropna().round(2).to_csv("modified/siuksles_surinktos.csv",index=False)

Calculating the score

In [381]:
import os

files = os.listdir("modified")
files_main = [i.split(".")[0] for i in files]

In [382]:
df = pd.read_csv("modified/"+files[0]).rename(columns={"y":files_main[0]})
for i,j in enumerate(files[1:]):
    df = pd.merge(df,pd.read_csv("modified/"+j),on=["x","city"],how="outer").rename(columns={"y":files_main[i+1]})

In [383]:
trans = [2,10]
water = [3,8,9]
air = [5,6,7]
trash = [4]

categories = [trans,water,air,trash]

In [385]:
df.iloc[:,[2,3,10]] =  -1 * df.iloc[:,[2,3,10]]
df = df.groupby(['city'], sort=False).apply(lambda x: x.ffill())

In [386]:
grouped = df.groupby("x")

l=[grouped.get_group(x) for x in grouped.groups]

In [387]:
df_scores = []

for df in l:
    df_min_max = df.iloc[:,2:]
    df.iloc[:,2:] = abs(((df_min_max-df_min_max.min())/(df_min_max.max()-df_min_max.min()))-1)
    for j,i in enumerate(categories):
        df.iloc[:,i] = df.iloc[:,i] * (25/len(i))
    df = df.round(2)
    df_scores.append(df.iloc[:,0:11])

C:\Users\dovyd\anaconda3\lib\site-packages\pandas\core\indexing.py:1851: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


In [390]:
pd.concat(df_scores).to_csv("kpi/indicators_scores.csv",index=False)

Algorithm for recalculating the weights and calculating the ranks

In [1016]:
df = pd.read_csv("kpi/categories.csv")
weights = [25,25,25,25]
df.iloc[:,[2,3,4,5]] = df.iloc[:,[2,3,4,5]] * pd.Series(weights,index = df.columns[[2,3,4,5]]) / 25 * 100 / sum(weights)
df["total"]= df.iloc[:,[2,3,4,5]].sum(axis=1)

In [934]:
df_ranks = df.copy()
df_ranks.iloc[:,2:] = df_ranks.iloc[:,1:].groupby("x").rank(ascending=False)